<a href="https://colab.research.google.com/github/samancha/nlp-master/blob/main/mod5/NLP_mod5_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data loading and preprocessing.

For some context, I've download the data and pushed it into my google drive or locally. I've developed both in colab and vs code, while pushing the notebook back and forth when doing heavier computation commands.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
! pip install transformers[torch]


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, TrainingArguments, Trainer
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
import nltk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import classification_report
# Download the stopwords and tokenizer from nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time
import datetime

seed_val = 42
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
print("gpu avail: ", torch.cuda.is_available())
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



Preprocessing data is essentially to getting optimal model results. I used a modular approach and created a function to process each "review" row in the data. I first began with tokenizing the text which better helped identify sets of words and distinguish puntuation more properly. I also lowered all the words and removing the following: punctuation, stopwords, and duplicate words. I was hesitant with removing duplicate words, but I was satisfied with the results I was able to produce.  

In [8]:
def preprocess_text(text):
    # Convert text to lowercase
    words = word_tokenize(text)

    # Convert words to lowercase
    words = [word.lower() for word in words]

    # Remove punctuation from words
    words = [word for word in words if word.isalnum()]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Remove duplicate words
    unique_words = list(dict.fromkeys(words))

    # Join the words back into a string
    text = ' '.join(unique_words)

    return text


I created a label column based on the sentiment, so that it would be friendlier to process. I also added the processed reviews to the data frame for future usage.

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/archive.zip')
df['label'] = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
df['processed_review'] = df['review'].apply(preprocess_text)
df.reset_index(drop=True)
display(df.head())

,review,sentiment,label,processed_review
0,One of the other reviewers has mentioned that ...,positive,1,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,positive,1,wonderful little production br filming techniq...
2,I thought this was a wonderful way to spend ti...,positive,1,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,0,basically family little boy jake thinks zombie...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,petter mattei love time money visually stunnin...


# Text tokenization and conversion to BERT input features.

In [10]:
# Load the pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

Initializing processed reviews as inputs and associated labels

In [11]:
inputs = df.processed_review.values
labels = df.label.values
print("Train data size ", len(inputs))

Train data size  50000


In [12]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in inputs:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        max_length = 64,           # Pad & truncate all sentences.
                        padding='max_length',
                        truncation=True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

print('Original: ', inputs[0])
print('Token IDs:', input_ids[0])
print('Tokenized:', tokenizer.decode(input_ids[0][0]))
print('Attention_mask', attention_masks[0])

Original:  one reviewers mentioned watching 1 oz episode hooked right exactly happened br first thing struck brutality unflinching scenes violence set word go trust show faint hearted timid pulls punches regards drugs sex hardcore classic use called nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em home many aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements never far would say main appeal due fact goes shows dare forget pretty pictures painted mainstream audiences charm romance mess around ever saw nasty surreal could ready watched developed taste got accustomed levels graphic injustice crooked guards sold nickel inmates kill order get away well mannered middle class turned bitches lack street skills experience may become comfortable uncomfortable viewing thats touch darker side
Token IDs: tensor([[  101,  20

In [ ]:
# Creating tensors for input_ids and attention masks
bert_input_ids = torch.cat(input_ids, dim=0)
bert_attention_masks = torch.cat(attention_masks, dim=0)
bert_labels = torch.tensor(labels)

# Bert Input Features types
print(type(bert_input_ids))
print(type(bert_attention_masks))

# Model definition, training, and evaluation.

### Sequence Classifacation Model Definition

In [19]:
# Load BertForSequenceClassification, the pretrained BERT model and trying to train on GPU
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

### training test split data

Use train_test_split to split our data into train and validations after alreadying having the bert input_ids and attention masks. Using this function to create DataLoaders to use the data sets for further processing.  

In [20]:

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

def train_valid_split(input_ids, attention_masks, labels):
  # Use 80% for training and 20% for validation.
  train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                              random_state=2021, test_size=0.2)
  # Do the same for the masks.
  train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
                                              random_state=2021, test_size=0.2)

  print('example train_input:', train_inputs[0])
  print('example attention_mask', train_masks[0])
  train_labels = torch.tensor(train_labels)
  validation_labels = torch.tensor(validation_labels)

  # Create the DataLoader for our training set.
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

  # Create the DataLoader for our validation set.
  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  return train_dataloader, validation_dataloader

In [16]:
# Returning my two dataloaders
bert_train_dataloader, bert_validation_dataloader = train_valid_split(bert_input_ids, bert_attention_masks, bert_labels)

example train_input: tensor([  101,  2471,  3585,  8201,  7842,  5737, 10649,  2466,  7987,  7814,
         6463,  2614,  4289, 12991, 20857,  3689,  7315,  3370,  4125,  2991,
         2637,  2034,  2931,  2694,  2739,  8133, 12347,  5896,  3257,  2488,
         3459,  2453,  2979, 20327, 13109,  5714,  6508,  2428,  4276,  3947,
         2204,  4516,  3395,  2190,  2126,  2175,  4050,  2844,  2537,  5300,
         3185,  5450, 23105,  2214, 19857,  4246, 11973,   102,     0,     0,
            0,     0,     0,     0])
example attention_mask tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0])


<ipython-input-15-f24dbceedb77>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
<ipython-input-15-f24dbceedb77>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_labels = torch.tensor(validation_labels)


### Evaluation



Creating funtion to train model and capture training loops and loss calculations on multiple epochs.

In [23]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

def train_model(model, epochs, train_dataloader, validation_dataloader):
    optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8 )
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
    # defining lists of collected values
    loss_values = []
    eval_accs = []

    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        t0 = time.time()

        total_loss = 0
        model.train()

        for step, batch in enumerate(train_dataloader):

            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()

            # Perform a forward pass (evaluate the model on this training batch).
            # This will return the loss (rather than the model output) because we
            # have provided the `labels`.
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

            # The call to `model` always returns a tuple, so we need to pull the
            # loss value out of the tuple.
            loss = outputs[0]

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value
            # from the tensor.
            total_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(train_dataloader)

        # Store the loss value for plotting the learning curve.
        loss_values.append(avg_train_loss)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        print("Running Validation...")

        t0 = time.time()
        model.eval()

        eval_loss, eval_accuracy = 0, 0
        nb_eval_steps, nb_eval_examples = 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            # batch = tuple(t for t in batch)
            b_input_ids, b_input_mask, b_labels = batch

            with torch.no_grad():
                # Forward pass, calculate logit predictions.
                # This will return the logits rather than the loss because we have
                # not provided labels.
                # token_type_ids is the same as the "segment ids", which
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask)

            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = outputs[0]
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            # Calculate the accuracy for this batch of test sentences.
            tmp_eval_accuracy = flat_accuracy(logits, label_ids)
            # Accumulate the total accuracy.
            eval_accuracy += tmp_eval_accuracy
            # Track the number of batches
            nb_eval_steps += 1

        avg_eval_acc = eval_accuracy/nb_eval_steps
        print("  Accuracy: {0:.2f}".format(avg_eval_acc))
        print("  Validation took: {:}".format(format_time(time.time() - t0)))
        eval_accs.append(avg_eval_acc)

    print("")
    print("Training complete!")
    return loss_values, eval_accs

In [58]:
epochs = 3
loss_values, eval_accs = train_model(model, epochs, bert_train_dataloader, bert_validation_dataloader)
print(loss_values)
print(eval_accs)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 3 ========
Training...
  Batch    40  of    625.    Elapsed: 0:00:28.
  Batch    80  of    625.    Elapsed: 0:00:55.
  Batch   120  of    625.    Elapsed: 0:01:21.
  Batch   160  of    625.    Elapsed: 0:01:48.
  Batch   200  of    625.    Elapsed: 0:02:15.
  Batch   240  of    625.    Elapsed: 0:02:41.
  Batch   280  of    625.    Elapsed: 0:03:08.
  Batch   320  of    625.    Elapsed: 0:03:34.
  Batch   360  of    625.    Elapsed: 0:04:01.
  Batch   400  of    625.    Elapsed: 0:04:28.
  Batch   440  of    625.    Elapsed: 0:04:55.
  Batch   480  of    625.    Elapsed: 0:05:21.
  Batch   520  of    625.    Elapsed: 0:05:48.
  Batch   560  of    625.    Elapsed: 0:06:15.
  Batch   600  of    625.    Elapsed: 0:06:41.

  Average training loss: 0.25
  Training epcoh took: 0:06:58
Running Validation...
  Accuracy: 0.86
  Validation took: 0:00:34

======== Epoch 2 / 3 ========
Training...
  Batch    40  of    625.    Elapsed: 0:00:27.
  Batch    80  of    625.    Elaps

Testing set using accuracy, precision, recall, and F1-score metrics

Tried to compute metrics using the Trainer Class and Trainer Arguments but was not able to compile and set up Trainer properly. If I would continue to do this, I learned this was the more proper way too late into the assignment.
Parts to remember or implement is to use pipelines and Trainer in together to be able to create metrics dynamically.

# Sample movie review predictions and explanations.


I was not able to get the syntax to properly work but my goal was to interate over a few examples of simple reviews and I was expecting to get a prediction of 1 or 0 to determine the sentiment.

In [ ]:
examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was idiotic.',
    'The movie was terrible...'
]

sample_predictions = []
for batch in bert_validation_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch[i]


    # Perform inference
    with torch.no_grad():
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels).logits
    pred = torch.argmax(outputs, dim=1)
    sample_predictions.append(pred)

print(sample_predictions)